### Transit ridership panel data models

Script to estimate the models of MUNI bus and rail ridership at a TAZ level for the purpose of determining the effect of TNCs on transit ridership in San Francisco. 

gde 3.28.2020

In [ ]:
import pandas as pd
import numpy as np

from linearmodels.panel import PanelOLS

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

%matplotlib inline

In [ ]:
# read the data and structure the data
data = pd.read_csv('data/PanelEstimationFile.csv', thousands = ',')


In [ ]:
# calculate dervied fields 

# setting flags
data['BUS'] = np.where(data['MODE']=='BUS', 1, 0)
data['RAIL'] = np.where(data['MODE']=='RAIL', 1, 0)
data['YEAR_2015'] = np.where(data['YEAR']==2015, 1, 0)

# segmenting bus vs rail coefficients
data['TRIP_STOPS_DELIVERED_BUS'] = data['TRIP_STOPS_DELIVERED']*data['BUS']
data['TRIP_STOPS_DELIVERED_RAIL'] = data['TRIP_STOPS_DELIVERED']*data['RAIL']

data['AVG_TNC_SMOOTH_BUS'] = data['AVG_TNC_SMOOTH'] * data['BUS']
data['AVG_TNC_SMOOTH_RAIL'] = data['AVG_TNC_SMOOTH'] * data['RAIL']

data['BUS_TRIP_STOPS_COMPETING_BUS'] = data['BUS_TRIP_STOPS_COMPETING'] * data['BUS']

# combining BART and Caltrain
data['BART_CT_AVG_RIDE_SMOOTH'] = data['BART_AVG_RIDE_SMOOTH'] + data['CALTRAIN_AVG_RIDE_SMOOTH']


In [ ]:
# Set the index
data = data.set_index(['ID', 'YEAR'])
data.head()

In [ ]:
# match Greg's combined TAZ model

mod = PanelOLS.from_formula("np.log(1+AVG_RIDE) \
                            ~ np.log(1+ACCESS_30) \
                            + np.log(1+ROUTES) \
                            + np.log(1+TRIP_STOPS_DELIVERED_BUS) \
                            + np.log(1+TRIP_STOPS_DELIVERED_RAIL) \
                            + ONTIME5 \
                            + np.log(1+BART_CT_AVG_RIDE_SMOOTH) \
                            + np.log(1+BUS_TRIP_STOPS_COMPETING_BUS) \
                            + np.log(1+AVG_TNC_SMOOTH_BUS) \
                            + np.log(1+AVG_TNC_SMOOTH_RAIL) \
                            + YEAR_2015 \
                            + EntityEffects",
             data = data)

res = mod.fit()
print(res)